In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

cuda


In [3]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 10)
vgg16.to(device)
print(vgg16)

/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
# optimizer
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [5]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [6]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [7]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [8]:
import torch
import torch.nn.functional as F

def plurality_vote_prediction(model, images, transformations, device):
    """
    Given a batch of images and a list of transformations,
    compute:
    The original prediction f(x)
    The predictions for each transformed image f(T_i(x))
    Then, determine the mode (most frequent prediction) among the transformed predictions.
    If f(x) equals the mode, the prediction is accepted; otherwise, it is rejected.
    
    Returns:
      accepted_preds: a tensor of shape [batch_size] where accepted samples have their original prediction,
                      and rejected samples are marked with -1.
      accepted_mask: a boolean tensor of shape [batch_size] indicating which samples were accepted.
    """
    model.eval()
    
    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape [batch_size]
    
    # 2) Get predictions from each transformation (set S)
    transformed_preds_list = []
    for tf in transformations:
        # Apply the transformation on CPU (if required) and move back to device.
        x_tf = tf(images.cpu()).to(device)
        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)
        transformed_preds_list.append(preds_tf)
    
    # 3) Compute the mode (most frequent prediction) over the transformed predictions.
    #    Note: If there are no transformations, we simply use the original predictions.
    if len(transformed_preds_list) > 0:
        stacked_preds = torch.stack(transformed_preds_list, dim=0)  # shape: [num_transforms, batch_size]
        mode_preds = stacked_preds.mode(dim=0).values  # shape: [batch_size]
    else:
        mode_preds = original_preds
    
    # 4) Determine acceptance: accepted if original_preds equals mode_preds.
    accepted_mask = (original_preds == mode_preds)
    
    # 5) Create an output: for accepted samples, return original prediction; for rejected, set to -1.
    accepted_preds = original_preds.clone()
    accepted_preds[~accepted_mask] = -1
    
    return accepted_preds, accepted_mask

In [9]:
from tqdm import tqdm

def test_with_plurality_vote(model, loader, transformations, device):
    """
    Evaluates the model on 'loader' using PluVot:
      - For each batch, it computes the plurality-voted predictions.
      - A sample is accepted if the original prediction f(x) equals the mode of the transformed predictions.
      - Rejected samples are marked with -1.
    
    It prints per-batch acceptance counts and returns:
      - The accuracy among accepted samples.
      - The overall acceptance rate.
    """
    model.eval()
    total_samples = 0
    total_accepted = 0
    correct_accepted = 0
    
    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)
        
        # Get PluVot predictions and acceptance mask for the batch.
        accepted_preds, accepted_mask = plurality_vote_prediction(model, images, transformations, device)
        
        batch_size = labels.size(0)
        total_samples += batch_size
        batch_accepted = accepted_mask.sum().item()
        total_accepted += batch_accepted
        
        # Calculate accuracy among accepted samples
        if batch_accepted > 0:
            batch_correct = (accepted_preds[accepted_mask] == labels[accepted_mask]).sum().item()
            correct_accepted += batch_correct
        
        print(f"Batch {i} | Accepted: {batch_accepted}/{batch_size}")
    
    overall_acceptance_rate = 100.0 * total_accepted / total_samples
    accepted_accuracy = 100.0 * correct_accepted / total_accepted if total_accepted > 0 else 0.0
    
    print(f"\nOverall Acceptance Rate: {overall_acceptance_rate:.2f}%")
    print(f"Accuracy among Accepted Samples: {accepted_accuracy:.2f}%")
    
    return accepted_accuracy, overall_acceptance_rate


In [10]:
accepted_acc, acceptance_rate = test_with_plurality_vote(vgg16, testloader, transformations, device)
print(f"Plurality Vote Accuracy among accepted samples: {accepted_acc:.2f}%")
print(f"Overall Acceptance Rate: {acceptance_rate:.2f}%")

  0%|▎                                                                                     | 1/313 [00:01<06:46,  1.30s/it]

Batch 0 | Accepted: 22/32


  1%|▌                                                                                     | 2/313 [00:01<04:20,  1.20it/s]

Batch 1 | Accepted: 20/32


  1%|▊                                                                                     | 3/313 [00:02<03:24,  1.52it/s]

...

Batch 310 | Accepted: 15/32


100%|███████████████████████████████████████████████████████████████████████████████████▋| 312/313 [02:15<00:00,  2.35it/s]

Batch 311 | Accepted: 16/32


100%|████████████████████████████████████████████████████████████████████████████████████| 313/313 [02:15<00:00,  2.31it/s]

Batch 312 | Accepted: 10/16

Overall Acceptance Rate: 60.29%
Accuracy among Accepted Samples: 9.92%
Plurality Vote Accuracy among accepted samples: 9.92%
Overall Acceptance Rate: 60.29%
